In [0]:
#!pip install skyfield
#!pip install tzwhere
#!pip install geopy

In [0]:
from datetime import datetime
from geopy import Nominatim
from tzwhere import tzwhere
from pytz import timezone, utc

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.patches import Circle

from skyfield.api import Star, load, wgs84
from skyfield.data import hipparcos, mpc, stellarium
from skyfield.projections import build_stereographic_projection
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN

import warnings
warnings.filterwarnings("ignore")

In [0]:
def load_data():
    # load celestial data
    # de421 shows position of earth and sun in space
    eph = load('de421.bsp')

    # hipparcos dataset contains star location data
    with load.open(hipparcos.URL) as f:
        stars = hipparcos.load_dataframe(f)

    # And the constellation outlines come from Stellarium.  We make a list
    # of the stars at which each edge stars, and the star at which each edge
    # ends.

    url = ('https://raw.githubusercontent.com/Stellarium/stellarium/master'
           '/skycultures/modern_st/constellationship.fab')

    with load.open(url) as f:
        constellations = stellarium.parse_constellations(f)
        
    return eph, stars, constellations

In [0]:
# load celestial data
eph, stars, constellations = load_data()

In [0]:
def collect_celestial_data(location, when):
    # get latitude and longitude of our location 
    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode(location)
    lat, long = location.latitude, location.longitude
    
    # convert date string into datetime object
    dt = datetime.strptime(when, '%Y-%m-%d %H:%M')

    # define datetime and convert to utc based on our timezone
    timezone_str = tzwhere.tzwhere().tzNameAt(lat, long)
    local = timezone(timezone_str)

    # get UTC from local timezone and datetime
    local_dt = local.localize(dt, is_dst=None)
    utc_dt = local_dt.astimezone(utc)

    # load celestial data
    # eph, stars, constellations = load_data()

    # find location of earth and sun and set the observer position
    sun = eph['sun']
    earth = eph['earth']

    # define observation time from our UTC datetime
    ts = load.timescale()
    t = ts.from_datetime(utc_dt)

    # define an observer using the world geodetic system data
    observer = wgs84.latlon(latitude_degrees=lat, longitude_degrees=long).at(t)

    # define the position in the sky where we will be looking
    position = observer.from_altaz(alt_degrees=90, az_degrees=0)
    # center the observation point in the middle of the sky
    ra, dec, distance = observer.radec()
    center_object = Star(ra=ra, dec=dec)

    # find where our center object is relative to earth and build a projection with 180 degree view
    center = earth.at(t).observe(center_object)
    projection = build_stereographic_projection(center)
    field_of_view_degrees = 180.0

    # calculate star positions and project them onto a plain space
    star_positions = earth.at(t).observe(Star.from_dataframe(stars))
    stars['x'], stars['y'] = projection(star_positions)
    
    edges = [edge for name, edges in constellations for edge in edges]
    edges_star1 = [star1 for star1, star2 in edges]
    edges_star2 = [star2 for star1, star2 in edges]

    
    return stars, edges_star1, edges_star2

In [0]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from datetime import datetime


def create_star_chart(location, when, chart_size, max_star_size):
    stars, edges_star1, edges_star2 = collect_celestial_data(location, when)
    limiting_magnitude = 10
    bright_stars = (stars.magnitude <= limiting_magnitude)
    magnitude = stars['magnitude'][bright_stars]
    fig, ax = plt.subplots(figsize=(chart_size, chart_size))
    
    #use the night sky color code
    border = plt.Circle((0, 0), 1, color='#041A40', fill=True) 
    ax.add_patch(border)

    marker_size = max_star_size * 10 ** (magnitude / -2.5)

    ax.scatter(stars['x'][bright_stars], stars['y'][bright_stars],
               s=marker_size, color='white', marker='.', linewidths=0,
               zorder=2)
    # Draw the constellation lines.
    xy1 = stars[['x', 'y']].loc[edges_star1].values
    xy2 = stars[['x', 'y']].loc[edges_star2].values
    lines_xy = np.rollaxis(np.array([xy1, xy2]), 1)

    ax.add_collection(LineCollection(lines_xy, colors='#ffff', linewidths=0.15))

    horizon = Circle((0, 0), radius=1, transform=ax.transData)
    for col in ax.collections:
        col.set_clip_path(horizon)

    # other settings
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    plt.axis('off')
    when_datetime = datetime.strptime(when, '%Y-%m-%d %H:%M')
    plt.title(f"Observation Location: {location}, Time: {when_datetime.strftime('%Y-%m-%d %H:%M')}", loc='right', fontsize=10)
   

    plt.show()

In [0]:
location = 'Wellington'
when = '2023-06-21 00:00'
chart_size=12
max_star_size=100
create_star_chart(location, when, chart_size, max_star_size)

In [0]:
def generate_star_chart(locations, whens):
    for location in locations:
        for when in whens:
            chart_size = 12
            max_star_size = 100

            # generate the plot
            create_star_chart(location, when, chart_size, max_star_size)

            # save the plot
            when_datetime = datetime.strptime(when, '%Y-%m-%d %H:%M')
            filename = f"{location}_{when_datetime.strftime('%Y%m%d_%H%M')}.png"
            #plt.savefig(filename, dpi=300)
            #plt.savefig(filename, format='svg', dpi=1200)
            plt.close()

            # print confirmation message
            print(f"Plot saved for location {location} and time {when}")

In [0]:
import os
os.getcwd()

In [0]:
import os
def create_folders_and_save_plots(locations, whens):
    for location in locations:
        # Create a folder for each location
        folder_name = location.replace(" ", "_")
        #os.makedirs(folder_name, exist_ok=True)
        #os.chdir(folder_name)

        # Call the function to generate and save plots
        generate_star_chart([location], whens)

        #os.chdir("..")  # Go back to the parent directory

# Example usage:
locations = ['Seattle', 'New York', 'Paris']
whens = ['2023-06-21 00:00', '2023-09-23 00:00','2023-12-22 00:00']
create_folders_and_save_plots(locations, whens)

In [0]:

from geopy.geocoders import Nominatim

# Create a Nominatim geocoder object
geolocator = Nominatim(user_agent="myGeocoder")

# Geocode a location to obtain latitude and longitude coordinates
location = "Seattle"
address = geolocator.geocode(location)
latitude = address.latitude
longitude = address.longitude

# Print the latitude and longitude
print(f"Latitude: {latitude}")
print(f"Longitude: {longitude}")
